In [1]:
!pip install tensorflow opencv-python-headless twilio pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.7 MB/s eta 0:00:00


Load and Preprocess Data:

In [1]:
import pandas as pd
def load_data(recipes_dataset):
    return pd.read_csv('recipes_dataset.csv')

def preprocess_data(data):
    data_numeric = data.iloc[:, 1:]
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    return X, y

data = load_data('recipes_dataset.csv')
X, y = preprocess_data(data)


Build and Train the Model:

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split

X_train = np.array([[1], [2], [3], [4]])
y_train = np.array([5, 6, 7, 8])

X_train = X_train.reshape(X_train.shape[0], 1, 1)

X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)


model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')


history = model.fit(X_train, y_train, epochs=50, batch_size=32)

model.save('ai_kitchen_model.h5')


Epoch 1/50
1/1 [==============================] - 5s 5s/step - loss: 43.6231
Epoch 2/50
1/1 [==============================] - 0s 12ms/step - loss: 43.3763
Epoch 3/50
1/1 [==============================] - 0s 11ms/step - loss: 43.1309
Epoch 4/50
1/1 [==============================] - 0s 12ms/step - loss: 42.8854
Epoch 5/50
1/1 [==============================] - 0s 12ms/step - loss: 42.6385
Epoch 6/50
1/1 [==============================] - 0s 13ms/step - loss: 42.3885
Epoch 7/50
1/1 [==============================] - 0s 10ms/step - loss: 42.1338
Epoch 8/50
1/1 [==============================] - 0s 10ms/step - loss: 41.8731
Epoch 9/50
1/1 [==============================] - 0s 12ms/step - loss: 41.6047
Epoch 10/50
1/1 [==============================] - 0s 12ms/step - loss: 41.3273
Epoch 11/50
1/1 [==============================] - 0s 10ms/step - loss: 41.0395
Epoch 12/50
1/1 [==============================] - 0s 14ms/step - loss: 40.7397
Epoch 13/50
1/1 [==============================] - 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


 Utensil Detection and Management
Setup Utensil Detection:

In [10]:
import cv2
import numpy as np

model = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')
layer_names = model.getLayerNames()
output_layers_indices = model.getUnconnectedOutLayers()

output_layers = [layer_names[i - 1] for i in output_layers_indices.flatten()]

with open('coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

utensil_mapping = {
    47: "cup",
    48: "fork",
    49: "knife",
    50: "spoon",
    51: "bowl"
}
def detect_utensils(image):
    height, width, channels = image.shape
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    model.setInput(blob)
    outs = model.forward(output_layers)


    utensils_detected = []
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    for i in range(len(boxes)):
        if i in indexes:
            class_id = class_ids[i]
            print(f"Detected class ID: {class_id}")
            utensil_name = utensil_mapping.get(class_id, "Utensil")
            utensils_detected.append((utensil_name, boxes[i]))

    return utensils_detected


image_path = 'kitchen.jpg'
image = cv2.imread(image_path)

if image is None:
    print(f"Error: Could not load image from {image_path}. Check the file path and permissions.")
else:
    utensils = detect_utensils(image)
    print(utensils)


[]


Raw Material Management
Check Ingredients:

In [11]:
import pandas as pd
inventory = pd.read_csv('inventory.csv')

def check_ingredients(recipe_ingredients):
    missing_ingredients = []
    for ingredient, quantity in recipe_ingredients.items():
        if ingredient in inventory['ingredient'].values:
            available_quantity = inventory[inventory['ingredient'] == ingredient]['quantity'].values[0]
            if available_quantity < quantity:
                missing_ingredients.append(ingredient)
        else:
            missing_ingredients.append(ingredient)
    return missing_ingredients

recipe_ingredients = {'tomato': 2, 'onion': 1, 'chicken': 1}
missing = check_ingredients(recipe_ingredients)
print(f'Missing ingredients: {missing}')


Missing ingredients: ['tomato']


Send Notifications:

In [58]:
from twilio.rest import Client

account_sid = "AC74514e08dc2a430808942e96bb562fc5"
auth_token = "3646036d7ad8c6f2b0cbbd51d99b906b"

client = Client(account_sid, auth_token)

def send_notification(message, to_phone_number):
  message = client.messages.create(
        to="+919044029367",
        from_="+919044029367",
        body=message
    )


missing_ingredients_message = "You are missing the following ingredients: " + ", ".join(missing)
send_notification(missing_ingredients_message, "+919044029367")


TwilioRestException: HTTP 400 error: Unable to create record: Invalid 'To' Phone Number: +91904402XXXX